In [1]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.8 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 80

In [2]:
import gc
import torch
from vllm import LLM, SamplingParams
from datasets import load_dataset
from tqdm.auto import tqdm

2025-07-22 01:38:16.436920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753148296.782720      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753148296.883367      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 07-22 01:38:36 [__init__.py:244] Automatically detected platform cuda.


In [3]:
def generate_answers(model_id, dataset_name):
    
    dataset = load_dataset(dataset_name, split="test")
    
    def format(sample):
        return "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{}\n\n### Response:\n".format(sample["instruction"])
    
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dataset = dataset.map(lambda sample: {"prompt": format(sample)})
    
    llm = LLM(model=model_id, max_model_len=4096, dtype="float16", device=device_type)
    sampling_params = SamplingParams(temperature=0.8, top_p=0.95, min_p=0.05, max_tokens=4096)
    outputs = llm.generate(dataset["prompt"], sampling_params)

    answers = [output.outputs[0].text for output in outputs]
    dataset = dataset.add_column("answers", answers)
    
    print(f"Uploading results for {model_id}")
    dataset.push_to_hub(f"SkillRipper/{model_id.split('/')[-1]}-results")
    gc.collect()
    return dataset

In [4]:
model_ids = ['unsloth/Llama-3.2-3B-Instruct', 'SkillRipper/TwinLlama-3.2-3B-DPO', 'SkillRipper/TwinLlama-3.2-3B-Instruct']

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
for model_id in model_ids:
    generate_answers(model_id, "SkillRipper/llmtwin")

README.md:   0%|          | 0.00/418 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/614k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/72.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1899 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/211 [00:00<?, ? examples/s]

...

INFO 07-22 01:46:32 [config.py:841] This model supports multiple tasks: {'generate', 'embed', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 07-22 01:46:32 [config.py:1472] Using max model len 4096
INFO 07-22 01:46:32 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='SkillRipper/TwinLlama-3.2-3B-Instruct', speculative_config=None, tokenizer='SkillRipper/TwinLlama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=Observa

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]